## Iterators

Iteration is fundamental to data processing: programs apply computations to data series, from pixels to nucleotides. *If the data doesn’t fit in memory, we need to fetch the items lazily—one at a time and on demand*. That’s what an iterator does.  

Every standard collection in Python is iterable. *An iterable is an object that provides an iterator*, which Python uses to support operations like:
- `for` loops  
- List, dict, and set comprehensions  
- Unpacking assignments  
- Construction of collection instances  